# **Why JSON mode?**

You can guarantee that a model's generated output always adheres to a specific schema so that you receive consistently formatted responses. For example, you might have an established data schema that you use for other tasks. If you have the model follow the same schema, you can directly extract data from the model's output without any post-processing.

To specify the structure of a model's output, define a response schema, which works like a blueprint for model responses. When you submit a prompt and include the response schema, the model's response always follows your defined schema.

# **Objectives**

In this tutorial, you will learn how to use either OpenAI SDK or Vertex AI SDK in Python to generated structured outputs via the Llama 4 Maverick fully managed model on Vertex AI.
 See here for more info on using the [OpenAI SDK with Vertex](https://cloud.google.com/vertex-ai/generative-ai/docs/migrate/openai/overview#:~:text=The%20Chat%20Completions%20API%20works,the%20Google%20Gen%20AI%20SDK.), as well as recommendations on when to use OpenAI SDK vs. Vertex AI SDK.

We will use sentiment analysis as an example use case, you can replace it with a different structure that's right for you.

# **Setup and Relevant Links**
Llama on Vertex AI (fully managed): https://cloud.google.com/vertex-ai/generative-ai/docs/partner-models/llama. You may also need to accept the EULA to continue.

Official docs from Vertex on structured outputs/JSON mode with Llama coming soon.


# **Setup and Relevant Links **

In [1]:
import sys
import IPython

%pip install --upgrade --user --quiet google-genai
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [12]:
import os

project_id = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not project_id or project_id == "[your-project-id]":
    project_id = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

# run gcloud auth print-access-token from terminal to get this
access_token = ""

location = os.environ.get("GOOGLE_CLOUD_REGION", "us-east5")


Defining the format we want to return...

In [4]:
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "ARRAY",
        "items": {
            "type": "OBJECT",
            "properties": {
                "rating": {"type": "INTEGER"},
                "flavor": {"type": "STRING"},
                "sentiment": {
                    "type": "STRING",
                    "enum": ["POSITIVE", "NEGATIVE", "NEUTRAL"],
                },
                "explanation": {"type": "STRING"},
            },
            "required": ["rating", "flavor", "sentiment", "explanation"],
        },
    },
}


# **Now, generate JSON output**

First, with OpenAI's SDK...

In [13]:
import json
import openai
from openai import OpenAI

# Setup client
base_url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/endpoints/openapi"
client = openai.OpenAI(base_url=base_url, api_key=access_token)

prompt = """
  Analyze the following product reviews, output the sentiment classification, and give an explanation.

  - "Absolutely loved it! Best ice cream I've ever had." Rating: 4, Flavor: Strawberry Cheesecake
  - "Quite good, but a bit too sweet for my taste." Rating: 1, Flavor: Mango Tango
"""

response = client.chat.completions.create(
    model="meta/llama-4-maverick-17b-128e-instruct-maas",
    messages=[{"role": "user", "content": prompt}],
    response_format={"type": "json_object"},
    temperature=0.1
)

product_reviews = json.loads(response.choices[0].message.content)
print(product_reviews)


{'reviews': [{'text': "Absolutely loved it! Best ice cream I've ever had.", 'rating': 4, 'flavor': 'Strawberry Cheesecake', 'sentiment': 'Positive', 'explanation': "The reviewer uses the phrase 'Absolutely loved it' and states it's the 'Best ice cream I've ever had', indicating a very positive sentiment. The rating of 4 out of a presumed 5 is consistent with this positive sentiment."}, {'text': 'Quite good, but a bit too sweet for my taste.', 'rating': 1, 'flavor': 'Mango Tango', 'sentiment': 'Negative', 'explanation': "Although the reviewer starts with 'Quite good', they follow it with a negative statement 'but a bit too sweet for my taste', indicating a mixed sentiment. However, the rating of 1 suggests a strongly negative sentiment, which is inconsistent with the text. The sentiment classification based on the text would be 'Mixed' or 'Neutral', but given the low rating, it leans more towards being negative overall."}]}


Now with Vertex AI SDK

In [ ]:
from google.genai.types import GenerateContentConfig, Part, SafetySetting

from google import genai

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
MODEL_ID =  "meta/llama-4-maverick-17b-128e-instruct-maas"


In [ ]:
prompt = """
  Analyze the following product reviews, output the sentiment classification, and give an explanation.

  - "Absolutely loved it! Best ice cream I've ever had." Rating: 4, Flavor: Strawberry Cheesecake
  - "Quite good, but a bit too sweet for my taste." Rating: 1, Flavor: Mango Tango
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)
product_reviews: dict = response.parsed
print(product_reviews)

[[{'explanation': "The reviewer used the phrase 'Absolutely loved it' and stated it was the 'Best ice cream I've ever had', indicating a very positive sentiment despite the rating being less than 5.", 'flavor': 'Strawberry Cheesecake', 'rating': 4, 'sentiment': 'POSITIVE'}, {'explanation': "The reviewer described the product as 'Quite good', but expressed a negative aspect by stating it was 'a bit too sweet', aligning with the low rating given. The negative aspect outweighs the positive, leading to an overall negative sentiment.", 'flavor': 'Mango Tango', 'rating': 1, 'sentiment': 'NEGATIVE'}]]


# **Congrats and conclusion**

You've successfully built a sentiment analyzer leveraging structured outputs via Llama 4 Maverick using the OpenAI and/or Vertex AI SDK!

# **Cleanup**
You can perform the following cleanup to avoid incurring charges to your Google Cloud account for the resources used in this codelab:
*   To avoid unnecessary Google Cloud charges, use the Google Cloud console to  delete your project if you do not need it.
*   If you want to disable the APIs for Vertex AI, navigate to the Vertex AI API Service Details page and click Disable API and confirm.
